In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from vnstock import *
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import yfinance as yf
from math import sqrt
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [17]:
data_ticker = pd.read_excel("C:\\Users\\Chi Bao\\OneDrive\\Desktop\\Đồ án tốt nghiệp\\data_ticker.xlsx")

FI = ['GC=F','SI=F','CL=F','^GSPC','^DJI','^IXIC','^RUT','^FTSE','^N225','^NYA']
df = pd.DataFrame()
for symbol in FI:
    data = yf.download(symbol, start='2012-03-20', end='2023-09-06')
    df[symbol] = data['Adj Close']

column_mapping = {
    'GC=F': 'Gold',
    'SI=F': 'Silver',
    'CL=F': 'Crude Oil',
    '^GSPC': 'S&P 500',
    '^DJI': 'Dow Jones',
    '^IXIC': 'NASDAQ',
    '^RUT': 'Russell 2000',
    '^FTSE': 'FTSE 100',
    '^N225': 'Nikkei 225',
    '^NYA': 'NYSE'
}

df = df.rename(columns=column_mapping)
df.dropna()

returns_df = df.pct_change()*100
returns_df.columns = [col + '_return' for col in returns_df.columns]
returns_df = returns_df.dropna()

###

FI = ['VNINDEX','HNX','VN30','HNX30','UPCOM']
data = stock_historical_data('VNINDEX', '2012-03-19', '2023-09-05', "1D", "index")
d_new = data.set_index('time')
d_new['VNINDEX'] = data['close']
df1 = pd.DataFrame()
for symbol in FI:
    data = stock_historical_data(symbol, '2012-03-19', '2023-09-05', "1D", "index")
    data = data.set_index('time')
    df1[symbol] = data['close']
df1 = df1.dropna()

returns_df1 = df1.pct_change()*100
returns_df1.columns = [col + '_return' for col in returns_df1.columns]
returns_df1 = returns_df1.dropna()
returns_df1.head(42069)

###

lagreturn = pd.DataFrame()

for column in returns_df.columns:
    lagreturn[f'{column}_Lag1'] = returns_df[column].shift(1)
for column in returns_df.columns:
    lagreturn[f'{column}_Lag2'] = returns_df[column].shift(2)
lagreturn = lagreturn.dropna()

lagreturn1 = pd.DataFrame()
for column in returns_df1.columns:
    lagreturn1[f'{column}_Lag1'] = returns_df1[column].shift(1)
for column in returns_df1.columns:
    lagreturn1[f'{column}_Lag2'] = returns_df1[column].shift(2)
lagreturn1 = lagreturn1.dropna()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [18]:
def do_tre_vnstock(symbol, start_date, end_date, lags=10): 
    ts = stock_historical_data(symbol,start_date, end_date)
    ts = ts.dropna(subset=['close'])


    tsret = pd.DataFrame(index=ts.index)
    tsret['time']=ts['time']
    tsret["Today"] = ts["close"].pct_change()*100

    for i,x in enumerate(tsret["Today"]):
        if (abs(x) < 0.0001):
            tsret["Today"][i] = 0.0001

    for i in range(0, lags):
        tsret["Lag%s" % str(i+1)] = tsret["Today"].shift(i+1)

    tsret["Direction"] = np.sign(tsret["Today"])
    tsret = tsret.dropna()
    tsret = tsret.set_index('time')
    bpt = pd.merge(tsret, lagreturn, left_index=True, right_index=True)
    bpt1 = pd.merge(bpt, lagreturn1, left_index=True, right_index=True)
    bpt1.fillna(0, inplace=True)
    bpt1 = bpt1[(bpt1 != 0).all(axis=1)]
    return bpt1

In [21]:
#MODEL
list_i = []
list_return_price = []
list_rmse = []
list_mae =[]
list_mse = []
list_r2_train = []
list_r2_test = []

for i in data_ticker['ticker']:
  try:
    data = do_tre_vnstock(i, '2012-03-19', '2023-09-05', lags=10)

    X = data.loc[:, ~data.columns.isin(['Today'])]
    y = data["Today"]
    
    t = round(sqrt(len(X)))
    model = KNeighborsRegressor(n_neighbors=t)
    X_train, X_test,y_train, y_test=train_test_split(X,y,test_size=0.2,random_state=12, shuffle=False)
    model.fit(X_train,y_train)
    yhat_test = model.predict(X_test)
    r2_train = model.score(X_train, y_train)
    r2_test = model.score(X_test, y_test)
    mse = mean_squared_error(y_test, yhat_test, squared=True)
    rmse = mean_squared_error(y_test, yhat_test, squared=False)
    mae = mean_absolute_error(y_test, yhat_test)

    X_predict = [X.iloc[-1].values.tolist()]
    Y_predict = model.predict(X_predict)

    list_i.append(i)
    list_return_price.append(Y_predict)
    list_r2_train.append(r2_train)
    list_r2_test.append(r2_test)
    list_rmse.append(rmse)
    list_mse.append(mse)
    list_mae.append(mae)

    print('Stock:',i,f' Predict: {Y_predict.item():.2f}',f' R2_train: {r2_train:.2f}',f' R2_test: {r2_test:.2f}',
          f' RMSE: {mse:.2f}',f' RMSE: {mse:.2f}', f'MAE: {mae:.2f}')
  except Exception as e:
    print(f"Stock: {i} -Unpredictable!\n" f'{e}')
    continue

Stock: SSI  Predict: -0.01  R2_train: 0.15  R2_test: 0.03  RMSE: 4.56  RMSE: 4.56 MAE: 1.55
Stock: BCM  Predict: 0.41  R2_train: 0.10  R2_test: 0.01  RMSE: 1.16  RMSE: 1.16 MAE: 0.72
Stock: VHM  Predict: 0.17  R2_train: 0.15  R2_test: 0.08  RMSE: 4.55  RMSE: 4.55 MAE: 1.58
Stock: VIC  Predict: -0.14  R2_train: 0.13  R2_test: 0.04  RMSE: 4.02  RMSE: 4.02 MAE: 1.18
Stock: VRE  Predict: 0.06  R2_train: 0.13  R2_test: 0.13  RMSE: 2.80  RMSE: 2.80 MAE: 1.22
Stock: BVH  Predict: 0.49  R2_train: 0.16  R2_test: 0.17  RMSE: 1.15  RMSE: 1.15 MAE: 0.78
Stock: POW  Predict: 0.13  R2_train: 0.13  R2_test: 0.14  RMSE: 2.38  RMSE: 2.38 MAE: 1.11
Stock: GAS  Predict: 0.32  R2_train: 0.19  R2_test: 0.21  RMSE: 0.90  RMSE: 0.90 MAE: 0.73
Stock: ACB  Predict: 0.42  R2_train: 0.14  R2_test: 0.09  RMSE: 1.49  RMSE: 1.49 MAE: 0.84
Stock: BID  Predict: 0.49  R2_train: 0.14  R2_test: 0.09  RMSE: 2.51  RMSE: 2.51 MAE: 1.13
Stock: CTG  Predict: 0.34  R2_train: 0.15  R2_test: 0.15  RMSE: 2.11  RMSE: 2.11 MAE: 1.

In [22]:
KQ = pd.DataFrame({'Stock': list_i, 'Predict': list_return_price, 'R2_train': list_r2_train,'R2_test': list_r2_test, 'RMSE': list_mse,'MSE': list_rmse, 'MAE': list_mae,})
KQ.to_excel("Kết quả 20% test size.xlsx")
KQ.head(42069)

,Stock,Predict,R2_train,R2_test,RMSE,MSE,MAE
0,SSI,[-0.0056450792883396515],0.147133,0.029622,4.558910,2.135160,1.550687
1,BCM,[0.4139372688973208],0.103440,0.008449,1.163705,1.078752,0.716213
2,VHM,[0.17411606816405303],0.146274,0.081340,4.552930,2.133760,1.576588
3,VIC,[-0.13541150935497012],0.130086,0.043221,4.020404,2.005094,1.184668
4,VRE,[0.06372353370591803],0.128113,0.128662,2.799560,1.673188,1.223494
...,...,...,...,...,...,...,...
389,TDP,[0.11846754075616273],0.114994,0.068367,5.536356,2.352946,1.482631
390,NO1,[-0.30465610359872797],0.173462,-0.021473,11.931685,3.454227,2.605501
391,L10,[1.0197772748039828],0.102733,0.034094,34.997156,5.915839,4.880124
392,REE,[0.08095429048987501],0.130131,0.151362,1.542790,1.242091,0.879384
